# `GCDM-SBDD` Tutorial, Part 2 - Filtering Generated Molecules using PoseBusters

## Overview

`GCDM-SBDD` provides a robust means of filtering new 3D molecules once generated within a target protein pocket of interest. As an illustrative real-world example, one can do so using the following 3-step procedure:

1. Create a new script called `pb_screen_mols.py` within the output directory `experiments/4OZ2_1Z6_07_25_2024_12_00_00/`. Fill this script with the following template source code, and then customize this source code for your target complex of interest.

```python
"""Template source code for filtering generated molecules using PoseBusters."""
import argparse
import glob
import os
import shutil
import tempfile

import pandas as pd

from posebusters import PoseBusters
from rdkit import Chem


BUST_TEST_COLUMNS = [
    # chemical validity and consistency #
    "mol_pred_loaded",
    "mol_cond_loaded",
    "sanitization",
    # intramolecular validity #
    "all_atoms_connected",
    "bond_lengths",
    "bond_angles",
    "internal_steric_clash",
    "aromatic_ring_flatness",
    "double_bond_flatness",
    "internal_energy",
    # intermolecular validity #
    "minimum_distance_to_protein",
    "minimum_distance_to_organic_cofactors",
    "minimum_distance_to_inorganic_cofactors",
    "volume_overlap_with_protein",
    "volume_overlap_with_organic_cofactors",
    "volume_overlap_with_inorganic_cofactors",
    # interreceptor validity
    "protein-ligand_maximum_distance",
]


def df_split_mol_frags(mol_table: pd.DataFrame) -> pd.DataFrame:
    """Split the molecules in the DataFrame into SDF fragments.

    :param mol_table: Molecule table DataFrame.
    :return: Molecule table DataFrame with fragments.
    """
    new_rows = []
    for row in mol_table.itertuples():
        mols_preds = Chem.SDMolSupplier(str(row.mol_pred), removeHs=False)
        for mol_pred in mols_preds:
            new_row = row._asdict()
            new_row["mol_cond"] = row.mol_cond
            with tempfile.NamedTemporaryFile(suffix=".sdf", delete=False) as temp_pred:
                Chem.MolToMolFile(mol_pred, temp_pred.name)
                new_row["mol_pred"] = temp_pred.name
            new_rows.append(new_row)
    new_df = pd.DataFrame(new_rows)
    new_df["mol_true"] = None
    return new_df


def pb_screen_mols(
    input_molecule_dir: str,
    reference_protein_path: str,
    output_molecule_dir: str,
    bust_results_filepath: str,
):
    """
    Screen a directory of molecule SDF files using the PoseBusters validity metric.
    Only PoseBuster-valid molecule SDF files will be saved to the output directory.

    :param input_molecule_dir: Path to a directory containing SDF files of molecules to screen.
    :param reference_protein_path: Path to a reference protein PDB file.
    :param output_molecule_dir: Path to a directory to which to save PoseBuster-valid SDF files.
    :param bust_results_filepath: Path to which to save the PoseBusters results.
    """
    assert os.path.exists(
        input_molecule_dir
    ), f"Input molecule directory {input_molecule_dir} does not exist."
    assert os.path.exists(
        reference_protein_path
    ), f"Reference protein file {reference_protein_path} does not exist."
    os.makedirs(output_molecule_dir, exist_ok=True)

    # collect inference files

    inference_sdf_results = [
        item
        for item in glob.glob(os.path.join(input_molecule_dir, "*"))
        if item.endswith(".sdf")
    ]
    assert inference_sdf_results, f"No SDF files found in {input_molecule_dir}."
    dataset_protein_files = [
        reference_protein_path for _ in range(len(inference_sdf_results))
    ]
    mol_table = pd.DataFrame(
        {
            "mol_pred": [item for item in inference_sdf_results if item is not None],
            "mol_true": None,
            "mol_cond": [item for item in dataset_protein_files if item is not None],
        }
    )
    mol_frags_table = df_split_mol_frags(mol_table)

    # screen molecules

    buster = PoseBusters(config="dock", top_n=None)
    bust_results = buster.bust_table(mol_frags_table, full_report=False)
    bust_results.to_csv(bust_results_filepath, index=False)
    print(
        f"PoseBusters results for input molecule directory {input_molecule_dir} saved to {bust_results_filepath}."
    )

    # filter invalid molecules

    bust_results_df = pd.read_csv(bust_results_filepath)
    bust_results_df["valid"] = bust_results_df[BUST_TEST_COLUMNS].all(axis=1)
    valid_mol_indices = bust_results_df[bust_results_df["valid"] == 1].index
    valid_mol_paths = mol_frags_table.iloc[valid_mol_indices]["mol_pred"]
    for valid_mol_path in valid_mol_paths:
        shutil.move(
            valid_mol_path,
            os.path.abspath(
                os.path.join(output_molecule_dir, os.path.basename(valid_mol_path))
            ),
        )
    print(f"PoseBuster-valid molecules saved to {output_molecule_dir}.")


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--input_molecule_dir",
        type=str,
        default=os.path.join(
            "experiments", "4OZ2_1Z6_07_25_2024_12_00_00", "outputs_07_25_2024_12_00_00"
        ),
        help="Path to a directory containing SDF files of molecules to screen.",
    )
    parser.add_argument(
        "--reference_protein_path",
        type=str,
        default=os.path.join(
            "experiments", "4OZ2_1Z6_07_25_2024_12_00_00", "4OZ2_without_1Z6.pdb"
        ),
        help="Path to a reference protein PDB file.",
    )
    parser.add_argument(
        "--output_molecule_dir",
        type=str,
        default=os.path.join(
            "experiments", "4OZ2_1Z6_07_25_2024_12_00_00", "pb_valid_outputs_07_25_2024_12_00_00"
        ),
        help="Path to a directory to which to save PoseBuster-valid SDF files.",
    )
    parser.add_argument(
        "--bust_results_filepath",
        type=str,
        default=os.path.join(
            "experiments",
            "4OZ2_1Z6_07_25_2024_12_00_00",
            "outputs_07_25_2024_12_00_00",
            "bust_results.csv",
        ),
        help="Path to which to save the PoseBusters results.",
    )
    args = parser.parse_args()

    pb_screen_mols(
        args.input_molecule_dir,
        args.reference_protein_path,
        args.output_molecule_dir,
        args.bust_results_filepath,
    )
```

2. Create a separate version of your initial input PDB file after manually removing the 3D coordinates of the ligand within your targeted binding pocket (e.g., `1Z6`, represented as residue `501` within chain `A`), and store this new file at `experiments/4OZ2_1Z6_07_25_2024_12_00_00/4OZ2_without_1Z6.pdb`.

3. Run this molecule filtering script via `python3 experiments/4OZ2_1Z6_07_25_2024_12_00_00/pb_screen_mols.py`.

The filtered output directory `experiments/4OZ2_1Z6_07_25_2024_12_00_00/pb_valid_outputs_07_25_2024_12_00_00/` should now be populated with approximately 260 [`PoseBusters`](https://github.com/maabuu/posebusters)-validated 3D molecules after (on average) 190 are found to be invalid by the [`PoseBusters` software suite](https://github.com/maabuu/posebusters). These filtered molecules can now be inspected manually (or with external tools) for final validation (e.g., prior to wet lab experiments).

Congrats! You've successfully generated a (filtered) batch of 3D molecules for your target protein pocket of interest. 🚀

## Wrapping up

Have any additional questions about filtering new 3D molecules generated using `GCDM-SBDD` within target protein pockets? [Create a new issue](https://github.com/BioinfoMachineLearning/GCDM-SBDD/issues/new/choose) on our [GitHub repository](https://github.com/BioinfoMachineLearning/GCDM-SBDD). We would be happy to work with you.